# Discretized Hamiltonian

We compute here the eigenvalues $\varepsilon_n$ and eigenfunctions $\psi_n(z)$ of an MTI slab which is infinitely extended along the in-plane directions $x$ and $y$ and which has a finite thickness $d$ along the out-of-plane direction $z$.
The three-dimensional MTI Hamiltonian can be decomposed as 
$$
    h_{MTI} = A \partial^2_z + B \partial_z + C \,, 
$$
where the derivative part is 
$$
	A = 
	\begin{bmatrix}
		(B_1-D_1) & 0 & 0 & 0 \\[5pt]
		%
		0 & (B_1-D_1) & 0 & 0 \\[5pt]
		%
		0 & 0 & -(B_1+D_1) & 0 \\[5pt]
		%
		0 & 0 & 0 & -(B_1+D_1)
	\end{bmatrix} \,,
    \qquad
	B = 
	\begin{bmatrix}
		0 & 0 & -i A_1 & 0 \\[5pt]
		%
		0 & 0 & 0 & i A_1  \\[5pt]
		%
		-i A_1 & 0 & 0 & 0 \\[5pt]
		%
		0 & i A_1 & 0 & 0 
	\end{bmatrix} \,,     
$$
and the constant term is given by
$$
    C =  
	\begin{bmatrix}
		\varepsilon_0 + \mu_0 + \Lambda & 0 & 0 & A_2 k_-\\[5pt]
		%
		0 & \varepsilon_0 + \mu_0 - \Lambda & A_2 k_+ & 0 \\[5pt]
		%
		0 & A_2 k_- & \varepsilon_0 - \mu_0 + \Lambda & 0 \\[5pt]
		%
		A_2 k_+ & 0 & 0 & \varepsilon_0 - \mu_0 - \Lambda \\[5pt]
	\end{bmatrix} \,.
$$
In the basis of states $\{ \ket{+,\uparrow}, \ket{+,\downarrow}, \ket{-,\uparrow}, \ket{-,\downarrow} \}$, the three matrices $A,B$ and $C$ can be expressed as
$$
\begin{gathered}
    A = B_1 \lambda_z \sigma_0 - D_1 \lambda_0 \sigma_0 \,, \\[5pt]
    %
    B = -i A_1 \lambda_x \sigma_z \,, \\[5pt]
        %
    C = \epsilon_0 \lambda_0 \sigma_0 + \Lambda \lambda_0 \sigma_z + A_2 \left( k_x \lambda_x  \sigma_x + k_y \lambda_x  \sigma_y \right) + \mu_0 \lambda_z \sigma_0 \,,
\end{gathered}
$$
where $\lambda_{x,y,z}$ and $\sigma_{x,y,z}$ are Pauli matrices acting in the parity and spin subspaces, respectively.

In [19]:
import numpy as np

# Hamiltonian parameters
params=dict(C = -0.0068, D1 = 1.3, D2 = 19.6, A1 = 2.2, A2 = 4.1, M = 0.28, B1 = 10, B2 = 56.6)

# Set of Pauli matrices for spin
sigma0 = np.array([[1, 0], [0, 1]])
sigmaX = np.array([[0, 1], [1, 0]])
sigmaY = np.array([[0, -1j], [1j, 0]])
sigmaZ = np.array([[1, 0], [0, -1]])


# Set of Pauli matrices for pseudo-spin
lambda0 = np.array([[1, 0], [0, 1]])
lambdaX = np.array([[0, 1], [1, 0]])
lambdaY = np.array([[0, -1j], [1j, 0]])
lambdaZ = np.array([[1, 0], [0, -1]])


# Identity matrix
Id = np.kron(tau0,sigma0)

# Function defining the Matrices A,B,C
def ABCmat(kx, ky, L, C = -0.0068, D1 = 1.3, D2 = 19.6, A1 = 2.2, A2 = 4.1, M = 0.28, B1 = 10, B2 = 56.6):

    k = np.sqrt(kx**2+ky**2); e0 = C + D2*k**2; m0 = M - B2*k**2

    #matrix A
    Amat = B1*np.kron(lambdaZ, sigma0) - D1*np.kron(lambda0, sigma0)

    # matrix B
    Bmat = -1j*A1*np.kron(lambdaX, sigmaZ)

    # matrix C
    Cmat = e0*np.kron(lambda0, sigma0) + L*np.kron(lambda0, sigmaZ) + A2*(kx*np.kron(lambdaX, sigmaX) + ky*np.kron(lambdaX, sigmaY)) + m0*np.kron(lambdaZ, sigma0)

    return [Amat,Bmat,Cmat]


# Function building the 3D MTI Hamiltonian from A,B,C
def hMTI(kx, ky, kz, L, C = -0.0068, D1 = 1.3, D2 = 19.6, A1 = 2.2, A2 = 4.1, M = 0.28, B1 = 10, B2 = 56.6):

    # build A,B,C matrices
    [Amat, Bmat, Cmat] = ABCmat(kx, ky, L, C=C, D1=D1, D2=D2, A1=A1, A2=A2, M=M, B1=B1, B2=B2)

    return -np.power(kz,2)*Amat + 1j*kz*Bmat + Cmat

Using the *finite difference approximations*, the spatial derivatives of the wavefunction $\psi \equiv \psi(z)$ can be approximated in the following way

- $ \partial_z f(z) \approx \bigl\lbrack f(z + \Delta z) − f(z − \Delta z) \bigr\rbrack / 2 \Delta z $, 

- $ \partial^2_z f(z) \approx \bigl\lbrack f(z + \Delta z) - 2f(z) + f(z − \Delta z) \bigr\rbrack / \Delta z^2 $,	

and the energy eigenvalue problem $h_{MTI} \psi = E \psi$ becomes
$$
\begin{split}
    h_{MTI} \psi(z) &= \Bigl\lbrack A \partial^2_z + B \partial_z + C \Bigr\rbrack \psi(z) = \\[5pt]
    %
    &= 
    A \frac{1}{\Delta z^2} 
    \Bigl\lbrack \psi(z + \Delta z) - 2\psi(z) + \psi(z − \Delta z) \Bigr\rbrack 
    +
    B \frac{1}{2 \Delta z}
    \Bigl\lbrack \psi(z + \Delta z) - \psi(z − \Delta z) \Bigr\rbrack 
    +
    C \psi(z) = \\[5pt]
    %
    &=
    \Bigl\lbrack \frac{A}{\Delta z^2} + \frac{B}{2 \Delta z} \Bigr\rbrack
    \psi(z + \Delta z)
    +
    \Bigl\lbrack C - \frac{2 A}{\Delta z^2} \Bigr\rbrack
    \psi(z)
    +
    \Bigl\lbrack \frac{A}{\Delta z^2} - \frac{B}{2 \Delta z} \Bigr\rbrack
    \psi(z - \Delta z) 
    =
    E \psi(z) \,.
\end{split}
$$
In the previous equation, we can recognize a tight-binding problem with on-site energy $h_0$ and hopping terms  $h_\pm$ given by 
$$
    h_0 = C - \frac{2 A}{\Delta z^2} \,,
    \qquad
    h_\pm = \frac{A}{\Delta z^2} \pm \frac{B}{2 \Delta z} \,.
$$ 
Given $N+1$ lattice points with spacing $\Delta z$ along the out-of-plane direction $z$ and assuming open boundary conditions at the extremities such that
$$
\begin{equation}
    h_- \psi (0) = 0 \,, \qquad h_+ \, \psi (N \Delta z) = 0 \,,
\end{equation}
$$
we can thus write the following set of equations for the confined based on the previous approximation
$$
\begin{equation}
    \begin{cases}
        h_0 \psi(0) +  h_+ \psi (\Delta y) = E  \psi(0) \,, \\[10pt]
        %
        h_- \psi((n-1)\Delta y) + h_0 \psi (n \Delta y) +  h_+ \psi ((n+1)\Delta y) = E \psi (n \Delta y) \,, \\[10pt]
        %
        h_- \psi ((N-1)\Delta y) + h_0 \psi (N \Delta y)  = E  \psi (N\Delta y) \,.    
    \end{cases}
\end{equation}
$$
Defining a discretized wavefunction on the lattice $$
    \Psi = 
    \begin{bmatrix}
    \psi(0) & \psi(\Delta z) & \dots & \psi(N \Delta z) 
    \end{bmatrix}^T \,,
$$
the energy eigenvalue problem can finally be rewritten as $H \Psi = E \Psi$ where 
$$
    H =
    \begin{bmatrix}
    h_0 & h_+ & 0 & & \dots & 0 \\[5pt]
    %
    h_- & h_0 & h_+ & 0 & \dots & 0 \\[5pt]
    %
    \vdots & & & & & \vdots \\[5pt]
    %
    0 & & & h_- & h_0 & h_+ \\[5pt]
    %
    0 & & & & h_- & h_0
    \end{bmatrix} \,.
$$
To ensure real energy eigenvalues the matrix $H$ must be hermitian: such conditions is satisfied if 
$$
    h_+^\dagger = 
    \frac{A^\dagger}{\Delta z^2} + \frac{B^\dagger}{2 \Delta z} 
    =
    \frac{A}{\Delta z^2} - \frac{B}{2 \Delta z} = h_-  \,,
$$
which means that $A = A^\dagger$ and $B^\dagger = -B$.
It's worth noting that these conditions are always satisfied:
$$
\begin{gathered}
    A^\dagger = B_1^\star \lambda_z \sigma_0 - D_1^\star \lambda_0 \sigma_0 = B_1 \lambda_z \sigma_0 - D_1 \lambda_0 \sigma_0 = A \,, \\[5pt]
    %
    B^\dagger = i A_1^\star \lambda_x \sigma_z = i A_1 \lambda_x \sigma_z = - B \,,
\end{gathered}
$$
which are always satisfied for $A_1, B_1$ and $D_1$ real.

In [40]:
from scipy.linalg import ishermitian
from scipy.sparse import coo_matrix, bmat
from scipy.sparse.linalg import eigs, eigsh


# Build the tigh-binding hamiltonian (Nlat=lattice points, dZ=lattice spacing)
def TBham(Nlat, dZ, kx, ky, L, C = -0.0068, D1 = 1.3, D2 = 19.6, A1 = 2.2, A2 = 4.1, M = 0.28, B1 = 10, B2 = 56.6):

    # build A,B,C matrices
    [Amat, Bmat, Cmat] = ABCmat(kx, ky, L, C=C, D1=D1, D2=D2, A1=A1, A2=A2, M=M, B1=B1, B2=B2)

    # on-site energy
    onsite = Cmat - 2./np.power(dZ,2)*Amat;

    # hopping energy
    hopping_plus = 1./np.power(dZ,2)*Amat + 1./(2.*np.power(dZ,2))*Bmat

    # hopping minus
    hopping_minus = np.conjugate(np.transpose(hopping_plus))

    # define a zero tight-binding matrix
    TBmat = [ [None for _ in range(Nlat) ] for _ in range(Nlat)]
		
	# populate the tight-binding matrix
    for i in range(Nlat):
		
        # onsite diagonal energy 
        TBmat[i][i] = onsite
        # hopping energy (n+1)
        if i+1 < Nlat: TBmat[i][i+1] = hopping_plus
        # hopping energy (n-1)
        if i-1 > -1: TBmat[i][i-1] = hopping_minus

    return bmat(TBmat)


# Compute energy and wavefunctions in the MTI slab
def 



(12, 12)
